In [1]:
pip install yahoofinancials

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
from yahoofinancials import YahooFinancials
import datetime
import json

In [3]:
file_path = f'data/yahoo/tickers.json'
with open(file_path, 'r') as json_file:
    tickers = json.load(json_file)

In [4]:
file_path = f'data/yahoo/merged_selected_financials.json'
with open(file_path, 'r') as json_file:
    yahoo_financials = json.load(json_file)

In [5]:
min_timestamp = datetime.datetime.now().timestamp()
max_timestamp = datetime.datetime(1970, 1, 1).timestamp()

for ticker in tickers:
    for statement in yahoo_financials[ticker]['annual_income']:
        for timestamp in statement:
            timestamp = int(timestamp)
            if timestamp < min_timestamp:
                min_timestamp = timestamp
            if timestamp > max_timestamp:
                max_timestamp = timestamp

print(f'Min timestamp: {datetime.datetime.fromtimestamp(min_timestamp)}')
print(f'Max timestamp: {datetime.datetime.fromtimestamp(max_timestamp)}')

Min timestamp: 2019-11-30 00:00:00
Max timestamp: 2023-10-31 00:00:00


In [6]:
def fetch_company_quotes(ticker, start_date, end_date):
    company = YahooFinancials(ticker)
    data = company.get_historical_price_data(start_date, end_date, 'daily')

    if 'prices' not in data[ticker]:
        return {}, False
    
    quotes = {}
    for quote in data[ticker]['prices']:
        quotes[str(quote['date'])] = {
            'adjclose': quote['adjclose'],
            'close': quote['close'],
            'high': quote['high'],
            'low': quote['low'],
            'open': quote['open'],
            'volume': quote['volume']
        }
        
    return quotes, True

In [7]:
def fetch_quotes(tickers, start_date, end_date):
    quotes = {}
    successes = {}
    for index, ticker in enumerate(tickers):
        print(f'[{index + 1}\t/ {len(tickers)}] Fetching {ticker} ... ', end='')

        company_quotes, success = fetch_company_quotes(ticker, start_date, end_date)
        successes[ticker] = success

        if not success:
            print('FAILED')
            continue

        quotes[ticker] = company_quotes
        successes[ticker] = True
        print('OK')

    return quotes, successes

In [8]:
start_date = str(datetime.date(2019, 11, 30))
end_date = str(datetime.date(2023, 10, 31))

quotes, successes = fetch_quotes(tickers, start_date, end_date)

[1	/ 504] Fetching A ... 

OK
[2	/ 504] Fetching AAL ... OK
[3	/ 504] Fetching AAP ... OK
[4	/ 504] Fetching AAPL ... OK
[5	/ 504] Fetching ABBV ... OK
[6	/ 504] Fetching ABC ... FAILED
[7	/ 504] Fetching ABMD ... OK
[8	/ 504] Fetching ABT ... OK
[9	/ 504] Fetching ACN ... OK
[10	/ 504] Fetching ADBE ... OK
[11	/ 504] Fetching ADI ... OK
[12	/ 504] Fetching ADM ... OK
[13	/ 504] Fetching ADP ... OK
[14	/ 504] Fetching ADSK ... OK
[15	/ 504] Fetching AEE ... OK
[16	/ 504] Fetching AEP ... OK
[17	/ 504] Fetching AES ... OK
[18	/ 504] Fetching AFL ... OK
[19	/ 504] Fetching AIG ... OK
[20	/ 504] Fetching AIZ ... OK
[21	/ 504] Fetching AJG ... OK
[22	/ 504] Fetching AKAM ... OK
[23	/ 504] Fetching ALB ... OK
[24	/ 504] Fetching ALGN ... OK
[25	/ 504] Fetching ALK ... OK
[26	/ 504] Fetching ALL ... OK
[27	/ 504] Fetching ALLE ... OK
[28	/ 504] Fetching AMAT ... OK
[29	/ 504] Fetching AMCR ... OK
[30	/ 504] Fetching AMD ... OK
[31	/ 504] Fetching AME ... OK
[32	/ 504] Fetching AMGN ... OK
[33	/ 504] Fetching AMP ... O

In [9]:
quotes_and_validities = {
    'quotes': quotes,
    'validities': successes
}

file_path = 'data/yahoo/quotes.json'
with open(file_path, 'w') as json_file:
    json.dump(quotes_and_validities, json_file, indent=4)